In [1]:
import json
import os
import xml.etree.ElementTree as ET
import pandas as pd

In [2]:
EMPTY_PAPER_DATA = {
    "perovskite_composition": None,
    "electron_transport_layer": None,
    "hole_transport_layer": None,
    "structure_pin_nip": None,
}
EMPTY_STABILITY_TEST = {
    "stability_type": None,
    "passivating_molecule": None,
    "humidity": None,
    "temperature": None,
    "time": None,
    "control_pce": None,
    "treated_pce": None,
    "control_voc": None,
    "treated_voc": None,
    "efficiency_control": None,
    "efficiency_tret": None
}

In [3]:
def get_json_for_passage(passage, relations, previous_json):
    concept_ids = set()
    for annotation in passage.findall(".//annotation"):
        node_id = annotation.get("id")
        var_name = annotation.find("infon[@key='type']").text
        concept_id = annotation.find("infon[@key='identifier']").text
        value = annotation.find("text").text
        value = concept_id if concept_id is not None else value
        if var_name == "perovskite_molecule": #due to an error in some of the annotations
            var_name = "passivating_molecule"
        if var_name in ["additive_molecule", "treatment_element", "control_element", "metal_contact"]: #irrelevant
            continue

        if var_name in ["perovskite_composition", "structure_pin_nip", "electron_transport_layer", "hole_transport_layer" ]:
            #in top level: composition, ETL, HTL, PIN-NIP,
            previous_json[var_name] = value
        elif node_id in relations:
            test_name = relations[node_id]
            if test_name not in previous_json:
                previous_json[test_name] = EMPTY_STABILITY_TEST.copy()
            previous_json[test_name][var_name] = value
        elif len(relations.keys()) == 0:
            if "test_1" not in previous_json:
                previous_json["test_1"] = EMPTY_STABILITY_TEST.copy()
            previous_json["test_1"][var_name] = value
            #in stability tests:
            #test type, passivator, PCE (control + treat), VOC (control + treat)
            #efficiency (treat, control), temp, time, humidity
        else:
            #assumes that all other possible data goes into the first stability test
            if "test_1" not in previous_json:
                previous_json["test_1"] = EMPTY_STABILITY_TEST.copy()
            previous_json["test_1"][var_name] = value

    return previous_json
    


In [4]:
def extract_papernum(root):
    first_text = root.find(".//text")
    full_text = first_text.text
    
    ##We want to extract article number from this format
    #Method: split by spaces and extract the last element in the list
    text_list = full_text.split()
    paper_num = text_list[-1]
    return paper_num

In [5]:
def parse_bioc_into_chunks(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    passages = root.findall('.//passage')
    data = []

    relations = {}
    test_names = set()
    for relation in root.findall(".//relation"):
        test_name = relation.find("infon[@key='type']").text
        if 'performance' in test_name: #irrelevant tests
            continue
        if test_name not in test_names:
            test_names.add(test_name)
        else:
            test_name = test_name + "_2"
        node_ids = [node.get("refid") for node in relation.findall("node")]
        for node_id in node_ids:
            relations[node_id] = test_name
    
    paper_num = extract_papernum(root)
    curr_json = EMPTY_PAPER_DATA.copy()
    for relation in root.findall('.//relation'):
        test_name = relation.find
    for i, passage in enumerate(passages):
        passage_text = passage.find('.//text').text
        row = { "id": f"{paper_num}_{i}", "text": passage_text, "memory": curr_json.copy() }
        curr_json = get_json_for_passage(passage, relations, curr_json)
        row['output'] = curr_json.copy()
        data.append(row)
    return data

In [6]:
bioc_dir = "data/biocs"
data = []
for filename in os.listdir(bioc_dir):
    if filename.endswith(".xml"):
        file_path = os.path.join(bioc_dir, filename)
        curr_paper_chunks = parse_bioc_into_chunks(file_path)
        data.extend(curr_paper_chunks)

df = pd.DataFrame(data)

In [9]:
df[['first_num', 'second_num']] = df['id'].str.split('_', expand=True)

# Step 2: Convert 'first_num' to numeric for proper sorting
df['first_num'] = df['first_num'].astype(int)

# Step 3: Group by 'first_num' and get the last row of each group
result = df.groupby('first_num', as_index=False).last()
result.to_csv('data/150_papers_json.csv', index=False)

In [7]:
df.to_csv("data/chunked_training.csv", index=False)

In [8]:
(df[df["id"].str.startswith("0_")].iloc[-1]["output"])

{'perovskite_composition': 'Cs 0.05 FA 0.85 MA 0.1 PbI 3',
 'electron_transport_layer': 'C60',
 'hole_transport_layer': '2PACz and Me-4PACz',
 'structure_pin_nip': 'PIN',
 'test_1': {'stability_type': 'ISOSL',
  'passivating_molecule': '4-chlorobenzenesulfonate (4Cl-BZS)',
  'humidity': None,
  'temperature': '65',
  'time': '1200',
  'control_pce': '24',
  'treated_pce': '26.9',
  'control_voc': None,
  'treated_voc': '1.18',
  'efficiency_control': None,
  'efficiency_tret': '95%'}}

In [9]:
paper_8 = df[df["id"].str.startswith("8_")]

In [10]:
paper_8.iloc[-1]["output"]

{'perovskite_composition': None,
 'electron_transport_layer': 'Tin Oxide',
 'hole_transport_layer': 'Spiro-OMeTAD',
 'structure_pin_nip': 'NIP',
 'test_1': {'stability_type': 'ISOSLT',
  'passivating_molecule': 'ethylammonium pyrene',
  'humidity': None,
  'temperature': '40',
  'time': '2000',
  'control_pce': '19.3',
  'treated_pce': '22.4',
  'control_voc': None,
  'treated_voc': '1.177',
  'efficiency_control': None,
  'efficiency_tret': '0.85',
  'efficiency_cont': '0.6'}}

In [11]:
example_1 = paper_8[paper_8['text'].str.contains("Supplementary Text S3")]

In [12]:
example_1[""].iloc[0]

KeyError: ''

In [99]:
example_1.to_csv("data/chunked_example.csv", index=False)